In [148]:
from deck import full_euchre_deck
from play_round import PlayRound, find_winners_vectorized
from dealer import Dealer
from branch_calc import compute_set_difference
import pandas as pd
import numpy as np

In [149]:
game = Dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands5 = [game.hand1, game.hand2, game.hand3, game.hand4]
hands5
# first_play = [game.hand1[0], game.hand1[1], game.hand1[2], game.hand1[3], game.hand1[4]]

[array([[-11,   0],
        [  0, 130],
        [  0, 100],
        [-13,   0],
        [  0, -13]]),
 array([[ 12,   0],
        [ 13,   0],
        [  0, -12],
        [-12,   0],
        [-14,   0]]),
 array([[   0,  120],
        [  -9,    0],
        [   0,   90],
        [   9,    0],
        [   0, -135]]),
 array([[  0,  -9],
        [ 10,   0],
        [  0, 110],
        [  0, -10],
        [  0, 140]])]

In [150]:
score = []
round1_tricks = []

In [151]:
for i in range(5):
    round1 = PlayRound(hands=hands5, lead=0, card_play=i)
    round1_play = round1.play_round()
    round1_tricks.append(round1_play)
    score.append(find_winners_vectorized(leads=np.zeros(len(round1_play), dtype=int), tricks=round1_play))
score = np.concatenate(score)
round1_tricks = np.concatenate(round1_tricks)
round2_hands = compute_set_difference(len(round1_tricks)*[hands5], round1_tricks)
len(score), len(round2_hands)

(90, 90)

In [152]:
round2_score = []
round2_tricks = []
round3_hands = []
r2_full_res = []

In [153]:
for i, j in zip(round2_hands, score):
    for k in range(4):
        round2 = PlayRound(hands=i, lead=j, card_play=k)
        round2_play = round2.play_round()
        round2_tricks.append(round2_play)
        winners = find_winners_vectorized(leads=len(round2_play)*[j], tricks=round2_play)
        round2_score.append(winners)
        full_score = np.column_stack((len(round2_play)*[j], winners))
        r2_full_res.append(full_score)
        test = len(round2_play)*[i]
        round3_hands.append(test)
round2_score = np.concatenate(round2_score)
round2_tricks = np.concatenate(round2_tricks)
round3_hands = np.concatenate(round3_hands)
round3_hands_comp = compute_set_difference(round3_hands, round2_tricks)
r2_full_res = np.concatenate(r2_full_res)

In [154]:
len(round2_tricks), len(round2_score), len(round3_hands), len(round3_hands_comp), len(r2_full_res)

(3248, 3248, 3248, 3248, 3248)

In [155]:
round3_score = []
round3_tricks = []
round4_hands = []
r3_full_res = []

In [156]:
for h, i, j in zip(r2_full_res, round3_hands_comp, round2_score):
    for k in range(3):
        round3 = PlayRound(hands=i, lead=j, card_play=k)
        
        round3_play = round3.play_round()
        round3_tricks.append(round3_play)
        winners = find_winners_vectorized(leads=len(round3_play)*[j], tricks=round3_play)
        round3_score.append(winners)
        full_score = np.column_stack((len(round3_play)*[h], winners))
        r3_full_res.append(full_score)

        test = len(round3_play)*[i]
        round4_hands.append(test)

round3_score = np.concatenate(round3_score)
r3_full_res = np.concatenate(r3_full_res)
round3_tricks = np.concatenate(round3_tricks)
round4_hands = np.concatenate(round4_hands)
round4_hands_comp = compute_set_difference(round4_hands, round3_tricks)

In [157]:
round4_score = []
round4_tricks = []
round5_hands = []
r4_full_res = []

In [158]:
for h, i, j in zip(r3_full_res, round4_hands_comp, round3_score):
    for k in range(2):
        round4 = PlayRound(hands=i, lead=j, card_play=k)
        round4_play = round4.play_round()
        round4_tricks.append(round4_play)
        winners = find_winners_vectorized(leads=len(round4_play)*[j], tricks=round4_play)
        round4_score.append(winners)
        full_score = np.column_stack((len(round4_play)*[h], winners))
        r4_full_res.append(full_score)

        test = len(round4_play)*[i]
        round5_hands.append(test)

        
round4_score = np.concatenate(round4_score)
r4_full_res = np.concatenate(r4_full_res)
round4_tricks = np.concatenate(round4_tricks)
round5_hands = np.concatenate(round5_hands)
round5_hands_comp = compute_set_difference(round5_hands, round4_tricks)

In [159]:
len(round4_score), len(round4_tricks), len(round5_hands_comp)

(674152, 674152, 674152)

In [160]:
round5_score = []

In [161]:
for h, i, j in zip(r4_full_res, round5_hands_comp, round4_score):
    round5 = PlayRound(hands=i, lead=j, card_play=0)
    round5_play = round5.play_round()
    winners = find_winners_vectorized(leads=len(round5_play)*[j], tricks=round5_play)
    full_score = np.column_stack((len(round5_play)*[h], winners))
    round5_score.append(full_score)
round5_score = np.concatenate(round5_score)

In [162]:
len(round5_score)

674152

In [163]:
res_df = pd.DataFrame(round5_score, columns=['r1_winner', 'r2_winner', 'r3_winner', 'r4_winner', 'r5_winner'])

In [164]:
res_df.columns

Index(['r1_winner', 'r2_winner', 'r3_winner', 'r4_winner', 'r5_winner'], dtype='object')

In [165]:
cols = ['r1_winner', 'r2_winner', 'r3_winner', 'r4_winner', 'r5_winner']

for i in cols:
    res_df[f'{i}_team'] = res_df[i] % 2

In [166]:
res_df.columns

Index(['r1_winner', 'r2_winner', 'r3_winner', 'r4_winner', 'r5_winner',
       'r1_winner_team', 'r2_winner_team', 'r3_winner_team', 'r4_winner_team',
       'r5_winner_team'],
      dtype='object')

In [167]:
res_df['points'] = res_df['r1_winner_team'] + res_df['r2_winner_team'] + res_df['r3_winner_team'] + res_df['r4_winner_team'] + res_df['r5_winner_team']

In [168]:
res_df['winner'] = np.where(res_df['points']>2, 1, 0)

In [169]:
res_df['winner'].mean()

0.2458614674435439